# Environment Setup

## なにをするのか

**Analyze(データ分析)のプロセスに必要な環境の仕様と構築に合意することがゴールです。**

1. Analyze:データ分析
   * ビジネスゴールを達成するために現状手に入るデータが必要十分であるか、データ分析を行いレポートを作成するプロセス。
2. 環境の仕様
   * 開発環境の構成図、Pythonやライブラリのバージョンなど。
3. 合意
   * プロジェクト内だけでなく、DevOpsチームやIT監督者といった機械学習モデルのリリースに必要な協力者の合意。

**最重要の成果物は開発環境の構築手順書です。**

環境が構築されたPCやサーバーに対し「合意」することは難しいので、構築手順を書き起こして合意します。手順書はGitHubなどバージョン管理できるプラットフォーム上に置くのが好ましいです。

環境構築手順書の例: [Wantedly Engineering Handbook](https://docs.wantedly.dev/fields/dev-tools/codespaces)

## なぜやるのか

DevOpsチームに負担をかけることを防ぐためです。

DevOpsチームはただでさえソフトウェア開発で忙しいので、リソースを割いて機械学習モデルを組み込むのは大変です。**事前に機械学習モデルの開発環境について合意しておきリリース時の驚きを減らす**ことが、負担の軽減につながります。ひいては、プロダクション環境で機械学習モデルが動く確率を高められます。

プロダクションで推論しない機械学習モデルは車に搭載されていないエンジンみたいなもので、性能が良くても価値を生むことはありません。

## どうやるのか

1. プログラミング演習
   * Studio Labを使いCondaで開発環境を構築する手順を学びます。
2. コミュニケーション演習
   * 開発環境を構築する際に行うべきコミュニケーションを学びます。

オンプレミスやクラウドでJupyterLabを起動する環境を作るにはネットワークや権限の設計、構築が必要です。今回はそうした構築を行わず無料の環境でまず試すというシーンを想定しSageMaker Studio Labを使用します。

開発環境の構築には[Conda](https://docs.conda.io/en/latest/)を使用します。Condaはプロジェクトごとの開発環境(仮想環境)を作成、管理するためのオープンソースです。なおPython以外の言語の環境も構築できます。

* venv/virtualenvとの相違点
   * 仮想環境が作れる点は同じです。CondaはPCやサーバー本体に入っているPythonと独立したPythonの実行環境を構築できます。pyenvと近しいです。
* pip/pipenv/poetryとの相違点
   * Pythonパッケージのインストール、依存関係管理が行える点は同じです。CondaはPython本体を含めPythonパッケージ以外も扱えます。Condaとpipなどは併用が可能です。ただし、同じパッケージをCondaとpip両方でインストールしないよう注意が必要です。

CondaはいくつかのPythonのパッケージと一体となった配布形式があります。Anacondaは代表的な配布形式の1つであり、商用利用する際は有償になります。詳細は[利用規約](https://www.anaconda.com/terms-of-service)を参照してください。Studio LabではCondaを使用していますがAnacondaに含まれるPythonパッケージを利用していないため無償で利用できます。詳細は[GitHubのIssue](https://github.com/aws/studio-lab-examples/issues/100)を参照してください。

構築する環境の手順は次の通りです。

1. SageMaker Studio Labを起動
2. `git clone`でファイルを取得
3. Condaを使用しPythonとライブラリをインストールする
4. インストールしたライブラリを記録したファイルを作成する

![environment_setup_procedure.PNG](./images/environment_setup_procedure.PNG)

### プログラミング演習

#### 1. SageMaker Studio Labを起動

Amazon SageMaker Studio Labはメールアドレスのみで、無料で利用できるJupyter Lab環境です。次のような特色があります。

* 12時間のCPU、4時間のGPUが利用可能。
   * 超過した場合セッションが切断されランタイムを再度起動する必要がありますが、ファイルは静的ストレージに維持されます。
* 15GBの静的ストレージがあり、作業内容が保管される。
* Gitと統合されており、GUIでファイルのバージョン管理が可能。
* Notebookに加えてTerminalも利用可能。

利用方法については、[Amazon SageMaker Studio Lab の使い方](https://github.com/aws-studiolab-jp/awesome-studio-lab-jp/blob/main/README_usage.md)を参照してください。

##### JupyterLabをローカルで構築する場合

Studio Labが利用できない場合、次の手順でJupyterLab環境が構築できます。

1. [Minicondaのインストール](https://docs.conda.io/en/latest/miniconda.html)
2. Terminalから`ml-handson`環境を作成: `conda create -n ml-handson python=3.9 jupyterlab`
3. Terminalから`ml-handson`環境を有効化: `conda activate ml-handson`
4. TerminalからJupyter Labを起動: `jupyter lab`

#### 2. `git clone`でファイルを取得

ハンズオンのコンテンツをダウンロードするために、Gitを使用します(Gitの説明は後ほど行います)。まずTerminalを起動します。

![00_environment_setup_003.PNG](images/00_environment_setup_003.PNG)

次のコマンドを実行しましょう。

```
git clone https://github.com/aws-samples/aws-ml-enablement-handson.git
```

ファイルエクスプローラーから`aws-ml-enablement-handson`というフォルダが見えたら成功です。

##### フォルダ構成

本ハンズオンのディレクトリ構造は次のようになっています。

```
├── LICENSE
├── README.md          <- リポジトリの概要を記載した文書
├── environment.yml    <- ハンズオンの動作に必要なライブラリをまとめたファイル
├── data
│   ├── raw            <- 生データ
│   ├── interim        <- 中間データ
│   └── processed      <- 前処理済みデータ
│
├── docs               <- ドキュメント
│
├── models             <- 学習済みモデル、前処理用パイプライン
│
└── notebooks          <- Jupyter notebooks
```

ディレクトリの構成は、[cookiecutter-data-science](https://github.com/drivendata/cookiecutter-data-science)というテンプレートに沿っています。

##### Git

ファイルの管理にGitを使用しています。基本的なコマンドと意味は次のようになります。`commit`は変更を記録する単位であり、1回の`commit`の大きさはどれだけ変更を`add`で登録するかに依存します。通常、複数の目的が混在したり大きすぎる`commit`は好まれません。今回であれば、複数プロセスのJupyter Notebook、例えば`01_business_understanding.ipynb`と`02_analyze.ipynb`が同じ`commit`に含まれるのは好ましくありません。

* `git clone`: リモートのリポジトリからローカルにダウンロードする。
* `git add`: 変更内容を登録する。
* `git commit`: 変更内容を記録する。

リモートリポジトリがあれば、`git push`でリモートリポジトリに変更を反映できます。変更を反映することで他の人と変更内容を共有できます。他の人も変更していた場合、変更の競合を解決できるように、通常Gitのリポジトリに変更を加える際は専用のbranchを作成して作業します。

![00_environment_setup_002.PNG](./images/00_environment_setup_002.PNG)

branchの作成にはいくつかルールがありますが、基本的にはbranchを作成、branchをリモートリポジトリにpush、変更内容の確認/承認依頼(Pull Request)を作成、承認されたら反映(merge)を行います。


#### 3. Condaを使用しPythonとライブラリをインストールする

Condaでハンズオンのための仮想環境を作成しましょう。Terminalを起動して、次のコマンドを実行してください。今回のハンズオンでは`pandas`という表形式データを扱うためのライブラリと`scikit-learn`という機械学習モデルが簡単に構築できるライブラリを主に扱います。

```
conda create -n ml-handson python=3.9 pandas scikit-learn matplotlib ipython ipykernel
```

*ハンズオン資料作成時点では`matplotlib`がPython 3.10で動かないため3.9を指定しています。Python 3.9に特別の依存があるわけではありません*

実行が完了したら、作成した`ml-handson`の環境を有効にするためのコマンドを実行します。

```
conda activate ml-handson
```

Jupyter Notebook上で`ml-handson`が利用できるようにコマンドを実行します。Jupyter Notebook上のコードの実行に使用する仮想環境を「カーネル」と呼び、次のコマンドでは作成した仮想環境をカーネルとして登録しています。

```
ipython kernel install --user --name ml-handson
```

完了したら、Jupyter Notebookの右上にある虫の隣のボタンをクリックしカーネルを切り替えます(作成した仮想環境が反映されるまでに少し時間がかかります)。

![00_environment_setup_004.PNG](images/00_environment_setup_004.PNG)

ここから先の開発プロセスでは、実行前にカーネルがが`ml-handson`になっていることを確認してください。

#### 4. Export

環境にインストールしたPythonやライブラリをファイルに書き出すことで、環境構築の手順を記録します。次のコマンドでファイルへの書き出しができます。一般的にCondaの環境は`environment.yml`に記録します。(ここでは演習用に名前を変えています)。

In [1]:
!conda env export -n ml-handson > environment_handson.yml

`environment_handson.yml`を開くと、インストールした覚えのないライブラリも記録されていると思います。依存ライブラリが記録されるためです。

In [2]:
from pathlib import Path


print(Path("environment_handson.yml").open().read())

name: ml-handson
channels:
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=2_gnu
  - alsa-lib=1.2.3.2=h166bdaf_0
  - asttokens=2.0.5=pyhd8ed1ab_0
  - backcall=0.2.0=pyh9f0ad1d_0
  - backports=1.0=py_2
  - backports.functools_lru_cache=1.6.4=pyhd8ed1ab_0
  - brotli=1.0.9=h166bdaf_7
  - brotli-bin=1.0.9=h166bdaf_7
  - bzip2=1.0.8=h7f98852_4
  - ca-certificates=2022.5.18.1=ha878542_0
  - cycler=0.11.0=pyhd8ed1ab_0
  - dbus=1.13.6=h5008d03_3
  - decorator=5.1.1=pyhd8ed1ab_0
  - entrypoints=0.4=pyhd8ed1ab_0
  - executing=0.8.3=pyhd8ed1ab_0
  - expat=2.4.8=h27087fc_0
  - font-ttf-dejavu-sans-mono=2.37=hab24e00_0
  - font-ttf-inconsolata=3.000=h77eed37_0
  - font-ttf-source-code-pro=2.038=h77eed37_0
  - font-ttf-ubuntu=0.83=hab24e00_0
  - fontconfig=2.14.0=h8e229c2_0
  - fonts-conda-ecosystem=1=0
  - fonts-conda-forge=1=0
  - freetype=2.10.4=h0708190_1
  - gettext=0.19.8.1=h73d1719_1008
  - giflib=5.2.1=h36c2ea0_2
  - gst-plugins-base=1.20.2=hcf0ee16_0
  

#### Extra Tutorial

#### Extra-1. Jupyter Lab

[Jupyter Lab](https://jupyter.org/)はソースコードと実行結果を含むドキュメントを作成できるJupyter Notebookの高機能版です。オープンソースで開発されています。Jupyter Notebookではセルと呼ばれる単位で文書を作成していきます。実行済みのセルで作成した変数はそれより下の他のセルからも参照可能です。

* セルの追加
  * 上に追加: A、下に追加: B
* セルの削除
  * DD
* セルの種別の変更
  * ソースコード: Y
  * 文書: M
* セルの実行
  * 実行: Ctrl + Enter
  * 実行して下に移動: Shift + Enter

#### Extra-2. Python Tutorial

今後のプログラミング演習ではPythonを使用します。Pythonの文法はここでは詳しく触れませんので、Referenceの資料を参照してください。各大学から素晴らしい教材が公開されています。次に簡単な計算例を示します。

In [3]:
x = 2
y = 2
print(x + y)

4



### コミュニケーション演習

環境構築として次の手順を実施しました。作業を行う際、誰にどのような質問をすべきでしょうか。

1. SageMaker Studio Labを起動
2. `git clone`でファイルを取得
3. Condaを使用しPythonとライブラリをインストールする
4. インストールしたライブラリを記録したファイルを作成する


| 英語名             | 日本語名                   | 役割                                                                 |
|--------------------|----------------------------|----------------------------------------------------------------------|
| Product Manager    | プロダクトマネージャー     | ユーザー体験を向上させるために実装すべきソフトウェア機能を定義する。 |
| Business Analyst   | ビジネスアナリスト         | 解決すべきビジネス上の問題を定義する                                 |
| Data Analyst       | データアナリスト           | データの可視化と分析で問題を定量的に特定する。                       |
| Architect          | アーキテクト               | ソフトウェアアーキテクチャ全体を設計する。                           |
| DevOps Engineer    | DevOpsエンジニア           | ソフトウェアの開発・運用プロセスを自動化する。                       |
| Software Engineer  | ソフトウェアエンジニア     | ソフトウェアの開発を行う。                                           |
| Operator           | 業務担当者                 | ソフトウェアを利用し業務を行う。                                     |
| System Admin       | システム管理者             | サービスの挙動を監視する。                                           |
| IT Auditor         | IT監督者                   | システムや会社全体の権限管理や監査を行う。                           |
| Data architect     | データアーキテクト         | データを管理する基盤を設計、運用する。                               |
| Domain Expert      | 業務有識者                 | 深い業務知識を持ちデータの意味やあるべき状態について定義する。       |
| Data Engineer      | データエンジニア           | 機械学習モデルに入力可能なデータと特徴を作成する。                   |
| Data Scientist     | データサイエンティスト     | 機械学習モデルを開発する。                                           |
| ML Engineer        | 機械学習エンジニア         | 機械学習モデルを本番環境にデプロイ可能な形式に変換する。             |
| ML Operator        | 業務担当者(機械学習側)     | 推論結果に基づき業務を行いつつ、フィードバックを与える。             |
| Model risk Manager | 機械学習モデルリスク管理者 | 機械学習モデルの推論の挙動を監視する。                               |
| MLOps Engineer     | MLOpsエンジニア            | 機械学習モデルの開発・運用プロセスを自動化する。                     |
| AI/ML Architect    | AI/MLアーキテクト          | 機械学習モデル開発に必要なアーキテクチャ全体を設計する。             |


#### 質問例

* IT Auditor
   * 必要十分なスペックのPCが用意可能か。
   * インストール可能なソフトウェアに制限はあるか。プロキシなどネットワークアクセスに制約はあるか。
   * 環境構築を行う上で制限を解除する必要がある場合、どのように申請すればよいか。
* Architect
   * 現在の開発でPythonは利用しているか。
   * 利用している場合、使用しているPythonのバージョンは何か。コーディングルールなどの開発標準があれば共有してほしい。
   * 利用していない場合、参考になりそうな既存の開発標準あるか。
   * 開発環境として利用してよい環境(AWSアカウントなど)。
* DevOps Engineer
   * ソースコード管理にどのようなサービスを利用しているか(GitHubなど)。Data Scientistでも利用してよいか。
   * これまで利用しているか。セットアップ方法にルールなどあるか聞いたほうが良いでしょう。開発ルールがあるなら合わせておく必要があるためです。

## Next Step

[Business Understanding](01_business_understanding.ipynb)


## References

1. Anaconda. [Terms of Service](https://www.anaconda.com/terms-of-service)
2. Greg Markovich. [Miniconda vs Anaconda | The Similarities and Differences](https://linuxnetmag.com/miniconda-vs-anaconda/). 2020.
3. Pythonの教材
   * 岡崎直観. [Python早見帳](https://chokkan.github.io/python/index.html). 2020.
   * 三谷純. [Python ゼロからはじめるプログラミング](https://mitani.cs.tsukuba.ac.jp/book_support/python/). 2021.
   * 東京大学 数理・情報教育研究センター. [Pythonプログラミング入門](https://utokyo-ipp.github.io/). 2020.
